# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [1]:
import helper

# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [2]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les Г©tats-unis est gГ©nГ©ralement froid en juillet , et il gГЁle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [2]:
import collections

english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [3]:
import project_tests as tests
from keras.preprocessing.text import Tokenizer

def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    x_tokenized = tokenizer.texts_to_sequences(x)
    
    return (x_tokenized, tokenizer)

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

Using TensorFlow backend.


{'dog': 5, 'a': 1, 'won': 6, 'brown': 7, 'by': 11, 'prize': 8, 'lazy': 9, 'lexicography': 4, 'jumps': 10, 'quick': 3, 'short': 13, 'over': 14, 'jove': 12, 'my': 15, 'is': 16, 'the': 2, 'sentence': 18, 'study': 19, 'of': 20, 'fox': 17, 'this': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [2, 3, 7, 17, 10, 14, 2, 9, 5]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [11, 12, 15, 3, 19, 20, 4, 6, 1, 8]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [21, 16, 1, 13, 18]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [4]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences

def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """

    if length is None:
        length = len(sorted(x, key=lambda s: len(s))[-1])
    
    x_padded = pad_sequences(x, length, padding='post')
    
    return x_padded

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [ 2  3  7 17 10 14  2  9  5]
  Output: [ 2  3  7 17 10 14  2  9  5  0]
Sequence 2 in x
  Input:  [11 12 15  3 19 20  4  6  1  8]
  Output: [11 12 15  3 19 20  4  6  1  8]
Sequence 3 in x
  Input:  [21 16  1 13 18]
  Output: [21 16  1 13 18  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [5]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

print('Data Preprocessed')

Data Preprocessed


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gap between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [6]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [9]:
from keras.layers import GRU, Input, Dense, TimeDistributed
from keras.models import Model
from keras.optimizers import Adam

def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    # define layers
    inp = Input(shape=input_shape[1:])
    gru = GRU(units=750, return_sequences=True)(inp)
    dense = TimeDistributed(Dense(units=french_vocab_size*2, activation='relu'))(gru)
    out = Dense(units=french_vocab_size, activation='softmax')(dense)
    
    # compose a model
    model = Model(inputs=inp, outputs=out)
    
    # compile the model
    optimizer = Adam(lr=.001, clipvalue=.5)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
   
    return model

tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(tmp_x.shape, preproc_french_sentences.shape[1],
                                len(english_tokenizer.word_index) + 1,
                                len(french_tokenizer.word_index) + 1)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=200, epochs=10, validation_split=0.2, verbose=2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
85s - loss: 1.3638 - acc: 0.6240 - val_loss: 1.0177 - val_acc: 0.6743
Epoch 2/10
84s - loss: 0.9013 - acc: 0.7056 - val_loss: 0.8758 - val_acc: 0.7011
Epoch 3/10
84s - loss: 0.7233 - acc: 0.7588 - val_loss: 0.6997 - val_acc: 0.7596
Epoch 4/10
84s - loss: 0.6253 - acc: 0.7898 - val_loss: 0.5741 - val_acc: 0.8122
Epoch 5/10
84s - loss: 0.5535 - acc: 0.8141 - val_loss: 0.5229 - val_acc: 0.8292
Epoch 6/10
84s - loss: 0.5042 - acc: 0.8295 - val_loss: 0.4905 - val_acc: 0.8350
Epoch 7/10
84s - loss: 0.4835 - acc: 0.8340 - val_loss: 0.4844 - val_acc: 0.8437
Epoch 8/10
84s - loss: 0.4537 - acc: 0.8436 - val_loss: 0.4341 - val_acc: 0.8506
Epoch 9/10
84s - loss: 0.4237 - acc: 0.8523 - val_loss: 0.3981 - val_acc: 0.8640
Epoch 10/10
84s - loss: 0.4076 - acc: 0.8572 - val_loss: 0.6278 - val_acc: 0.7847
new jersey est parfois calme en mois de mai et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [12]:
from keras.layers.embeddings import Embedding

def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    # define layers
    inp = Input(shape=input_shape[1:])
    emb = Embedding(750, english_vocab_size)(inp)
    gru = GRU(units=750, return_sequences=True)(emb)
    dense = TimeDistributed(Dense(units=french_vocab_size*2, activation='relu'))(gru)
    out = Dense(units=french_vocab_size, activation='softmax')(dense)
    
    # compose a model
    model = Model(inputs=inp, outputs=out)
    
    # compile the model
    optimizer = Adam(lr=.001, clipvalue=.5)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

tests.test_embed_model(embed_model)

# Reshaping the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])

# Train the neural network
emb_model = embed_model(tmp_x.shape, preproc_french_sentences.shape[1],
                                len(english_tokenizer.word_index) + 1,
                                len(french_tokenizer.word_index) + 1)
emb_model.fit(tmp_x, preproc_french_sentences, batch_size=200, epochs=10, validation_split=0.2, verbose=2)

# Print prediction(s)
print(logits_to_text(emb_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
93s - loss: 0.9810 - acc: 0.7566 - val_loss: 0.3812 - val_acc: 0.8747
Epoch 2/10
91s - loss: 0.3128 - acc: 0.8948 - val_loss: 0.2645 - val_acc: 0.9086
Epoch 3/10
91s - loss: 0.2399 - acc: 0.9172 - val_loss: 0.2305 - val_acc: 0.9210
Epoch 4/10
91s - loss: 0.2113 - acc: 0.9263 - val_loss: 0.2123 - val_acc: 0.9267
Epoch 5/10
91s - loss: 0.1947 - acc: 0.9313 - val_loss: 0.1976 - val_acc: 0.9310
Epoch 6/10
91s - loss: 0.1846 - acc: 0.9341 - val_loss: 0.1979 - val_acc: 0.9307
Epoch 7/10
91s - loss: 0.1795 - acc: 0.9357 - val_loss: 0.1952 - val_acc: 0.9322
Epoch 8/10
91s - loss: 0.1740 - acc: 0.9373 - val_loss: 0.1872 - val_acc: 0.9353
Epoch 9/10
92s - loss: 0.1676 - acc: 0.9391 - val_loss: 0.1850 - val_acc: 0.9354
Epoch 10/10
92s - loss: 0.1653 - acc: 0.9394 - val_loss: 0.1869 - val_acc: 0.9346
new jersey est parfois calme en l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [15]:
from keras.layers import Bidirectional

def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """

    # define layers
    inp = Input(shape=input_shape[1:])
    bd = Bidirectional(GRU(units=750, return_sequences=True), merge_mode='sum')(inp)
    dense = TimeDistributed(Dense(units=french_vocab_size*2, activation='relu'))(bd)
    out = Dense(units=french_vocab_size, activation='softmax')(dense)
    
    # compose a model
    model = Model(inputs=inp, outputs=out)
    
    # compile the model
    optimizer = Adam(lr=.001, clipvalue=.5)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

tests.test_bd_model(bd_model)

# Reshaping the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bidir_model = bd_model(tmp_x.shape, preproc_french_sentences.shape[1],
                                len(english_tokenizer.word_index) + 1,
                                len(french_tokenizer.word_index) + 1)
bidir_model.fit(tmp_x, preproc_french_sentences, batch_size=200, epochs=10, validation_split=0.2, verbose=2)

# Print prediction(s)
print(logits_to_text(bidir_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
159s - loss: 1.2066 - acc: 0.6537 - val_loss: 0.9372 - val_acc: 0.6873
Epoch 2/10
157s - loss: 0.8152 - acc: 0.7215 - val_loss: 0.7188 - val_acc: 0.7515
Epoch 3/10
157s - loss: 0.6404 - acc: 0.7784 - val_loss: 0.5771 - val_acc: 0.7999
Epoch 4/10
157s - loss: 0.5253 - acc: 0.8193 - val_loss: 0.4897 - val_acc: 0.8314
Epoch 5/10
157s - loss: 0.4641 - acc: 0.8380 - val_loss: 0.4599 - val_acc: 0.8424
Epoch 6/10
157s - loss: 0.4150 - acc: 0.8550 - val_loss: 0.4575 - val_acc: 0.8410
Epoch 7/10
157s - loss: 0.3825 - acc: 0.8648 - val_loss: 0.4665 - val_acc: 0.8317
Epoch 8/10
157s - loss: 0.3570 - acc: 0.8726 - val_loss: 0.3690 - val_acc: 0.8664
Epoch 9/10
157s - loss: 0.3488 - acc: 0.8752 - val_loss: 0.3322 - val_acc: 0.8868
Epoch 10/10
157s - loss: 0.3269 - acc: 0.8818 - val_loss: 0.3439 - val_acc: 0.8729
new jersey est parfois calme pendant l' automne et il et neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [36]:
from keras.layers import RepeatVector

def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """

    # define layers
    inp = Input(shape=input_shape[1:])
    gru_enc = GRU(units=750)(inp)
    rv = RepeatVector(output_sequence_length)(gru_enc)
    gru_dec = GRU(units=750, return_sequences=True)(rv)
    dense = TimeDistributed(Dense(units=french_vocab_size*2, activation='relu'))(gru_dec)
    out = Dense(units=french_vocab_size, activation='softmax')(dense)
    
    # compose a model
    model = Model(inputs=inp, outputs=out)
    
    # compile the model
    optimizer = Adam(lr=.001, clipvalue=.5)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

tests.test_encdec_model(encdec_model)

# Reshaping the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
enc_dec_model = encdec_model(tmp_x.shape, preproc_french_sentences.shape[1],
                                len(english_tokenizer.word_index) + 1,
                                len(french_tokenizer.word_index) + 1)
enc_dec_model.fit(tmp_x, preproc_french_sentences, batch_size=200, epochs=10, validation_split=0.2, verbose=2)

# Print prediction(s)
print(logits_to_text(enc_dec_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
214s - loss: 1.7564 - acc: 0.5681 - val_loss: 1.3388 - val_acc: 0.6168
Epoch 2/10
209s - loss: 1.2192 - acc: 0.6477 - val_loss: 1.1516 - val_acc: 0.6586
Epoch 3/10
209s - loss: 0.9993 - acc: 0.6869 - val_loss: 0.9019 - val_acc: 0.7080
Epoch 4/10
209s - loss: 0.8470 - acc: 0.7199 - val_loss: 0.8055 - val_acc: 0.7265
Epoch 5/10
209s - loss: 0.7369 - acc: 0.7449 - val_loss: 0.6994 - val_acc: 0.7537
Epoch 6/10
209s - loss: 0.6362 - acc: 0.7739 - val_loss: 0.5800 - val_acc: 0.7948
Epoch 7/10
210s - loss: 0.5687 - acc: 0.7963 - val_loss: 0.5731 - val_acc: 0.7921
Epoch 8/10
210s - loss: 0.4921 - acc: 0.8193 - val_loss: 0.4333 - val_acc: 0.8424
Epoch 9/10
209s - loss: 0.4627 - acc: 0.8298 - val_loss: 0.5162 - val_acc: 0.8148
Epoch 10/10
209s - loss: 0.4302 - acc: 0.8396 - val_loss: 0.6945 - val_acc: 0.7664
new jersey est parfois chaud au mois de mai et il est chaud en novembre <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [8]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
   
    # define layers
    inp = Input(shape=input_shape[1:])
    emb = Embedding(750, english_vocab_size)(inp)
    bd = Bidirectional(GRU(units=750, return_sequences=True))(emb)
    dense = TimeDistributed(Dense(units=french_vocab_size*2, activation='relu'))(bd)
    out = Dense(units=french_vocab_size, activation='softmax')(dense)
    
    # compose a model
    model = Model(inputs=inp, outputs=out)
    
    # compile the model
    optimizer = Adam(lr=.001, clipvalue=.5)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

tests.test_model_final(model_final)

print('Final Model Loaded')

Final Model Loaded


## Prediction (IMPLEMENTATION)

In [9]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences

def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """

    # train the neural network
    x = pad(x, y.shape[1])
    final_model = model_final(x.shape, y.shape[1],
                              len(x_tk.word_index) + 1,
                              len(y_tk.word_index) + 1)
    final_model.fit(x, y, batch_size=200, epochs=10, validation_split=0.2, verbose=2)
    
    model = final_model
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.argmax(x)] for x in y[0]]))

final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
183s - loss: 0.8135 - acc: 0.7971 - val_loss: 0.2767 - val_acc: 0.9107
Epoch 2/10
180s - loss: 0.1981 - acc: 0.9360 - val_loss: 0.1487 - val_acc: 0.9523
Epoch 3/10
180s - loss: 0.1235 - acc: 0.9602 - val_loss: 0.1200 - val_acc: 0.9613
Epoch 4/10
181s - loss: 0.0907 - acc: 0.9711 - val_loss: 0.0896 - val_acc: 0.9728
Epoch 5/10
180s - loss: 0.0721 - acc: 0.9773 - val_loss: 0.0808 - val_acc: 0.9756
Epoch 6/10
181s - loss: 0.0617 - acc: 0.9809 - val_loss: 0.0713 - val_acc: 0.9785
Epoch 7/10
181s - loss: 0.0515 - acc: 0.9839 - val_loss: 0.0727 - val_acc: 0.9790
Epoch 8/10
181s - loss: 0.0453 - acc: 0.9859 - val_loss: 0.0646 - val_acc: 0.9818
Epoch 9/10
181s - loss: 0.0405 - acc: 0.9875 - val_loss: 0.0634 - val_acc: 0.9819
Epoch 10/10
181s - loss: 0.0346 - acc: 0.9894 - val_loss: 0.0654 - val_acc: 0.9821
Sample 1:
il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a

## Submission
When you are ready to submit your project, do the following steps:
1. Ensure you pass all points on the [rubric](https://review.udacity.com/#!/rubrics/1004/view).
2. Submit the following in a zip file.
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.